In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
#!pip uninstall tensorflow -y
#!pip install tensorflow-gpu==2.9.0
#!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 -y

In [11]:
import tensorflow as tf
tf.__version__
tf.test.is_gpu_available()

True

In [110]:
import tensorflow as tf
import numpy as np
#from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Flatten
from tensorflow.keras.models import Sequential
from sklearn.metrics import r2_score

In [210]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [181]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

In [217]:
class model_(Model):
    def __init__(self):
        super().__init__()
        self.base = model
        (self.base).trainable = False
        self.int = Dense(5)
        self.classifier = Dense(1)
        self.dropout = Dropout(0.2)
    def call(self, inputs, train = None):
        output = self.base(inputs)[1]
        output = self.int(output)
        output = self.dropout(output, train)
        return self.classifier(output)

In [66]:
from datasets import load_dataset
dataset = load_dataset("glue", 'cola')

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [86]:
dataset_train = dataset["train"]
dataset_val = dataset["validation"]

In [91]:
tokenized_train = tokenizer(dataset_train["sentence"], return_tensors = "np", padding = True)
tokenized_test = tokenizer(dataset_val["sentence"], return_tensors = "np", padding = True)

In [117]:
tokenized_train = dict(tokenized_train)
tokenized_test = dict(tokenized_test)

In [111]:
train_labels = np.number(dataset_train["label"])
test_labels = np.number(dataset_test["label"])

In [218]:
model_bert = model_()

In [237]:
model_bert.compile(optimizer = "adam", loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics= ["accuracy"])
model_bert.fit(tokenized_train, train_labels, batch_size = 128, epochs = 5, validation_data = (tokenized_test, test_labels))

Epoch 1/5
67/67 [==============================] - 38s 324ms/step - loss: 0.5789 - accuracy: 0.6715 - val_loss: 0.5793 - val_accuracy: 0.7325
Epoch 2/5
67/67 [==============================] - 18s 275ms/step - loss: 0.5738 - accuracy: 0.6825 - val_loss: 0.5839 - val_accuracy: 0.7383
Epoch 3/5
21/67 [========>.....................] - ETA: 11s - loss: 0.5847 - accuracy: 0.6763

KeyboardInterrupt: 

In [220]:
y_pred = model_bert.predict(tokenized_test)

33/33 [==============================] - 6s 66ms/step


In [228]:
sum(test_labels)/len(test_labels)

0.6912751677852349